<p style="font-weight:bold;"> <span style="font-size: 36px"> Technical Margin Allocation</span> </p>

In [0]:
#!import "AocStructureTest"

# Workspace Initialization 

In [0]:
//define group of contract
var groupOfContract = "DT1.1";

In [0]:
//Define partition
    var args = new ImportArgs("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Actual);
    var partition = new PartitionByReportingNodeAndPeriod { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)),
                                                        ReportingNode = args.ReportingNode, 
                                                        Scenario = args.Scenario, 
                                                        Year = args.Year,
                                                        Month = args.Month };
    var partition2 = new PartitionByReportingNode { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)),
                                                        ReportingNode = args.ReportingNode, 
                                                        Scenario = args.Scenario};

In [0]:
var dt11 = new GroupOfInsuranceContract(){
    Portfolio = "DT1",
    Profitability = "P",
    LiabilityType = "LRC",
    AnnualCohort = 2020,
    Partition = partition.Id,
    ContractualCurrency =  "USD",
    //FunctionalCurrency =  "USD",
    LineOfBusiness = "ANN",
    ValuationApproach = "BBA",
    OciType ="Default" ,
    SystemName = "DT1.1",
    DisplayName = "DT1.1 OCI LRC PA 0.8"
};

In [0]:
var dnState = new DataNodeState {
            DataNode = "DT1.1",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partition.Id
        };

In [0]:
await DataSource.UpdateAsync(new[]{partition});
await DataSource.UpdateAsync(new[]{partition2});
await DataSource.UpdateAsync(new[]{dnState});
await DataSource.UpdateAsync(new[]{dt11});

In [0]:
var projectionConfiguration =
@"@@ProjectionConfiguration,,,,,,,,,,,
SystemName,DisplayName,Shift,TimeStep,,,,,,,,
P0,End of January,0,1,,,,,,,,
P1,End of February,0,2,,,,,,,,
P2,End of March,0,3,,,,,,,,
P3,End of April,0,4,,,,,,,,
P4,End of May,0,5,,,,,,,,
P5,End of June,0,6,,,,,,,,
P6,End of July,0,7,,,,,,,,
P7,End of August,0,8,,,,,,,,
P8,End of September,0,9,,,,,,,,
P9,End of October,0,10,,,,,,,,
P10,End of November,0,11,,,,,,,,
P11,End of December,0,12,,,,,,,,
P12,End of Year+1,12,12,,,,,,,,
P13,End of Year+2,24,12,,,,,,,,
P14,End of Year+3,36,12,,,,,,,,
P15,End of Year+4,48,12,,,,,,,,
P16,Year+5 to Year+10,60,60,,,,,,,,
P17,Year+10 to Year+15,120,60,,,,,,,,
P18,Year+15 to Year+20,180,60,,,,,,,,
P19,Years Over +20,240,9999,,,,,,,,";

var yieldCurve = 
@"@@YieldCurve,,,,,,
Currency,Year,Month,Values0,Values1,Values2,Values3
EUR,2020,3,0.008685495,0.008685495,0.018498471,0.019966839
CHF,2020,3,0.002,0.002,0.002,0.002
CHF,2021,3,0.1,0.1,0.1,0.1
USD,2021,3,0.005,0.005,0.005,0.005
USD,2020,12,0.002,0.002,0.002,0.002
USD,2020,3,0.002,0.002,0.002,0.002
USD,2020,1,0.002,0.002,0.002,0.002
GBP,2021,3,0.005,0.005,0.005,0.005
GBP,2020,3,0.002,0.002,0.002,0.002";

var currencies = 
@"@@Currency,,,,,,,,,,,
SystemName,DisplayName,,,,,,,,,,
USD,United States Dollar,,,,,,,,,,
CHF,Swiss Franc,,,,,,,,,,
DKK,Danish Krone,,,,,,,,,,
EUR,Euro,,,,,,,,,,
GBP,British Pound,,,,,,,,,,
HKD,Hong Kong Dollar,,,,,,,,,,
ITL,Italian Lira,,,,,,,,,,
PLN,Polish Zloty (since 01.01.95),,,,,,,,,,
SKK,Slovakian Krona,,,,,,,,,,";

var amountType = 
@"@@AmountType,,,,,,,,,,,
SystemName,DisplayName,Parent,Order,PeriodType,ExternalId0,ExternalId1,ExternalId2,ExternalId3,,,
PR,Premiums,,10,BeginningOfPeriod,PayablePR,ReceivablePR,RiPayablePR,RiReceivablePR,,,
CL,Claims,,20,EndOfPeriod,,,,,,,
NIC,Non Investment Component,CL,30,EndOfPeriod,PayableNIC,ReceivableNIC,RiPayableNIC,RiReceivableNIC,,,
ICO,Investment Component,CL,40,EndOfPeriod,PayableICO,ReceivableICO,,,,,
CDR,Credit Default Risk,CL,50,EndOfPeriod,,,,,,,
CE,Claim Expenses,CL,200,EndOfPeriod,,,,,,,
ALE,Allocated Loss Adjustment Expenses,CE,210,EndOfPeriod,,,,,,,
ULE,Unallocated Loss Adjustment Expenses,CE,220,EndOfPeriod,,,,,,,
AE,Attributable Expenses,,80,BeginningOfPeriod,,,,,,,
AEA,Aquisition,AE,90,BeginningOfPeriod,,,,,,,
AEM,Maintenance,AE,100,BeginningOfPeriod,,,,,,,
NE,Non Attributable Expenses,,110,BeginningOfPeriod,,,,,,,
AC,Attributable Commission,,120,BeginningOfPeriod,,,,,,,
ACA,Aquisition,AC,130,BeginningOfPeriod,,,,,,,
ACM,Maitenance,AC,140,BeginningOfPeriod,,,,,,,";

In [0]:
await Import.FromString(projectionConfiguration).WithType<ProjectionConfiguration>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(currencies).WithType<Currency>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(yieldCurve).WithType<YieldCurve>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(amountType).WithType<AmountType>().WithTarget(DataSource).ExecuteAsync();

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource)
         .DisableInitialization<RawVariable>()
         .DisableInitialization<IfrsVariable>());

# Test switch logic

In [0]:
public async Task CheckSwitchLogicAsync(IEnumerable<IfrsVariable> inputDataSet, Dictionary<AocStep,(double valueCsm, double valueLc)> csmLcSwitchBenchmark)
{
    //Save test input data
    await Workspace.UpdateAsync<IfrsVariable>(inputDataSet);
    
    //Set up import storage and test universe
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    var testUniverse = Scopes.ForStorage(testStorage).ToScope<IModel>();
    
    var identities = testUniverse.GetScopes<GetIdentities>(testStorage.DataNodesByImportScope[ImportScope.Primary].Where(dn => dn == groupOfContract)).SelectMany(s => s.Identities);
    var csm = testUniverse.GetScopes<AllocateTechnicalMargin>(identities, o => o.WithContext("C")).Where(x => Math.Abs(x.Value) > Precision);
    var lc = testUniverse.GetScopes<AllocateTechnicalMargin>(identities, o => o.WithContext("L")).Where(x => Math.Abs(x.Value) > Precision);
    //Clean up Workspace
    await Workspace.DeleteAsync<IfrsVariable>(await Workspace.Query<IfrsVariable>().ToArrayAsync());    
    
    var csmBenchmark = csmLcSwitchBenchmark.Where(x => Math.Abs(x.Value.valueCsm) > Precision).ToDictionary(x => x.Key, x => x.Value);
    var lcBenchmark = csmLcSwitchBenchmark.Where(x => Math.Abs(x.Value.valueLc) > Precision).ToDictionary(x => x.Key, x => x.Value);
        
        
    var errors = new List<string>();
    if(csm.Count() > csmBenchmark.Count()) 
    {
        var extraVariables = csm.Where(x => !csmBenchmark.Keys.Contains(new AocStep(x.Identity.AocType, x.Identity.Novelty)))
            .Select(x => $"AocType:{x.Identity.AocType}, Novelty:{x.Identity.Novelty}, EstimateType:{x.EstimateType}, Value:{x.Value}.");
        errors.Add( $"{nameof(AllocateTechnicalMargin)} scope for CSM has more non zero items than benchmark. Extra computed variables : \n {string.Join("\n", extraVariables)}." );
    }
    if(lc.Count() > lcBenchmark.Count()) 
    {
        var extraVariables = lc.Where(x => !lcBenchmark.Keys.Contains(new AocStep(x.Identity.AocType, x.Identity.Novelty)))
            .Select(x => $"AocType:{x.Identity.AocType}, Novelty:{x.Identity.Novelty}, EstimateType:{x.EstimateType}, Value:{x.Value}, ");
        errors.Add( $"{nameof(AllocateTechnicalMargin)} scope for LC has more non zero items compared to benchmark: \n {string.Join("\n", extraVariables)}." );
    }
    
    //Check ValueCsm
    foreach(var kvp in csmBenchmark)
    {
        var scopeSwitch = csm.SingleOrDefault(y => y.Identity.AocType == kvp.Key.AocType && y.Identity.Novelty == kvp.Key.Novelty);
        
        if(scopeSwitch == null)
        {
            errors.Add( $"Missing calculated scope for AocType and Novelty: {kvp.Key.AocType}, {kvp.Key.Novelty}." );
            continue;
        }
        
        if(Math.Abs(-1.0 * scopeSwitch.Value - kvp.Value.valueCsm) > Precision) 
            errors.Add( $"Values not matching for AocType {kvp.Key.AocType} and Novelty {kvp.Key.Novelty}. Scope: Csm {-1 * scopeSwitch.Value}; Benchmark: Csm {kvp.Value.valueCsm}." );
    }
    //Check ValueLc
    foreach(var kvp in lcBenchmark)
    {
        var scopeSwitch = lc.SingleOrDefault(y => y.Identity.AocType == kvp.Key.AocType && y.Identity.Novelty == kvp.Key.Novelty);
        
        if(scopeSwitch == null)
        {
            errors.Add( $"Missing calculated scope for AocType and Novelty: {kvp.Key.AocType}, {kvp.Key.Novelty}." );
            continue;
        }
        
        if(Math.Abs(scopeSwitch.Value - kvp.Value.valueLc) > Precision) 
            errors.Add( $"Values not matching for AocType {kvp.Key.AocType} and Novelty {kvp.Key.Novelty}. Scope: Lc {scopeSwitch.Value}; Benchmark: Lc {kvp.Value.valueLc}." );
    }
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
}

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfContract, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = -10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = 15.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CL", Novelty = "C", Value = 100.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","N"),(0d, 100d)},
        {new AocStep("IA","N"), (0d,  15d)},
        {new AocStep("EV","N"), (0d, 100d)},
        {new AocStep("CL","C"), (0d, 100d)},
        {new AocStep("EA","C"), (0d, -8.0)},
        {new AocStep("AM","C"), (0d, -153.5)},
        {new AocStep("EOP","C"),(0d, 153.5)},
    };



In [0]:
await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfContract, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = -10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = 15.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CL", Novelty = "C", Value = -500.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","N"),(0d, 100d)},
        {new AocStep("IA","N"), (0d,  15d)},
        {new AocStep("EV","N"), (0d, 100d)},
        {new AocStep("CL","C"), (285d, -215d)},
        {new AocStep("EA","C"), (8d, 0d)},
        {new AocStep("AM","C"), (-146.5d, 0d)},
        {new AocStep("EOP","C"),(146.5d, 0d)},
    };

await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfContract, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = -10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = -150.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","N"), (0d, 100d)},
        {new AocStep("IA","N"), (50d, -100d)},
        {new AocStep("EA","C"), (8d,   0d)},
        {new AocStep("AM","C"), (-29d,   0d)},
        {new AocStep("EOP","C"),(29d,   0d)},
    };

await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfContract, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "I", Value = 5010.0, EstimateType = "L", AmountType = null},
    basicIfrsVariable with {AocType = "MC", Novelty = "I", Value = -10.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "I", Value = -5015.0},
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = -100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = 10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = 150.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "N", Value = -45.0},
    basicIfrsVariable with {AocType = "CL", Novelty = "C", Value = -30.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","I"),(  0d, 5010.0)},
        {new AocStep("MC","I"), (  0d, -10d)},
        {new AocStep("IA","I"), (  0d, 10d)},
        {new AocStep("EV","I"), (  5d,-5010d)},
    
        {new AocStep("BOP","N"),(100d,   0d)},
        {new AocStep("IA","N"), (-100d,  50d)},
        {new AocStep("EV","N"), (  0d, -45d)},
    
        {new AocStep("CL","C"), ( 25d,  -5d)},
        {new AocStep("EA","C"), ( -8d,   0d)},
        {new AocStep("AM","C"), ( -11d,   0d)},
        {new AocStep("EOP","C"),( 11d,   0d)},
    };

await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);